In [1]:
!ldconfig /usr/lib64-nvidia
!ldconfig -p | grep libcud

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

	libcudnn_ops_train.so.8 (libc6,x86-64) => /lib/x86_64-linux-gnu/libcudnn_ops_train.so.8
	libcudnn_ops_train.so (libc6,x86-64) => /lib/x86_64-linux-gnu/libcudnn_ops_train.so
	libcudnn_ops_infer.so.8 (libc6,x86-64) => /lib/x86_64-linux-gnu/libcudnn_ops_infer.so.8
	libcudnn_ops_infer.so (libc6,x86-64) => /lib/x86_64-linux-gnu/libcudnn_ops_infer.so
	libcudnn_cnn_train.so.8 (libc6,x86-64) => /lib/x86_64-linux-gnu/libcudnn_cnn_train.so.8
	libcudnn_cnn_train.so (libc6,x86-64) => /lib/x86_64-linux-gnu/libcudnn_cnn_train.so
	libcud

In [4]:
import triton
import triton.language as tl
import torch


@triton.jit
def dummy_kernel(
    A,
    B,
    B_stride
):
    a = tl.num_programs(axis=0)
    b = tl.num_programs(axis=1)
    c = tl.num_programs(axis=2)

    id0 = tl.program_id(axis=0)
    id1 = tl.program_id(axis=1)
    id2 = tl.program_id(axis=2)

    # store num_programs in A
    tl.store(A+0, a)
    tl.store(A+1, b)
    tl.store(A+2, c)

    # store prograrm_id in B
    prg_index = id0 * (b * c) + id1 * c + id2
    B_base = B + B_stride * prg_index

    tl.store(B_base+0, id0)
    tl.store(B_base+1, id1)
    tl.store(B_base+2, id2)


In [3]:
torch.manual_seed(42)

grid = (4, 3, 2)
A = torch.zeros(len(grid), dtype=torch.float32, device="cuda")
B = torch.zeros(24, len(grid), dtype=torch.float32, device="cuda")

print(B.stride(0))

3


In [7]:
C = torch.zeros(grid, dtype=torch.float32, device="cuda")
print(C.stride(0))

6


In [5]:
dummy_kernel[grid](A, B, B.stride(0))
torch.cuda.synchronize()

print("program_ids", B)
print("num_programs", A)

program_ids tensor([[0., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 1.],
        [0., 2., 0.],
        [0., 2., 1.],
        [1., 0., 0.],
        [1., 0., 1.],
        [1., 1., 0.],
        [1., 1., 1.],
        [1., 2., 0.],
        [1., 2., 1.],
        [2., 0., 0.],
        [2., 0., 1.],
        [2., 1., 0.],
        [2., 1., 1.],
        [2., 2., 0.],
        [2., 2., 1.],
        [3., 0., 0.],
        [3., 0., 1.],
        [3., 1., 0.],
        [3., 1., 1.],
        [3., 2., 0.],
        [3., 2., 1.]], device='cuda:0')
num_programs tensor([4., 3., 2.], device='cuda:0')
